# Install Dependencies

## External Dependencies

In [1]:
!pip install inflect


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


## FrostAura Dependencies

In [2]:
import os
import sys

USE_LOCAL_CODE: bool = True # NOTE: When changing this configuration, be sure to restart the interpreter in order to have sys path updates reflect.
directories_to_move_up: int = 2
current_executing_path: str = os.getcwd()
current_executing_path = current_executing_path.replace('\\', '/')
root_path: str = '/'.join(current_executing_path.split('/')[:-directories_to_move_up])

# Clean up.
%pip uninstall frostaura -y
sys.path = [p for p in sys.path if p != root_path]

if USE_LOCAL_CODE:
    sys.path.append(root_path)
else:
    %pip install -U --no-cache-dir frostaura

Note: you may need to restart the kernel to use updated packages.


# Import Dependencies

## Logging

In [3]:
import logging

logging.getLogger().disabled = True

## Imports

In [4]:
from frostaura.engines import IAssetProjectionEngine
from frostaura.engines import SimpleAssetProjectionEngine
import pandas as pd
from frostaura.engines import IAssetValuationEngine
from frostaura.engines import FinvizAssetValuationEngine
from frostaura.engines import IAssetCalculationsEngine
from frostaura.engines import SimpleAssetCalculationsEngine
from frostaura.data_access import EasyEquitiesPersonalAssetDataAccess
from frostaura.data_access import IResourcesDataAccess
from frostaura.data_access import EmbeddedResourcesDataAccess
from frostaura.data_access import HtmlResourcesDataAccess
from frostaura.data_access import IPersonalAssetDataAccess
from frostaura.data_access import IPublicAssetDataAccess
from frostaura.data_access import YahooFinanceDataAccess
from frostaura.models import ValuationResult

# Usage

## Setup

In [5]:
html_data_access: IResourcesDataAccess = HtmlResourcesDataAccess()
resource_data_access: IResourcesDataAccess = EmbeddedResourcesDataAccess()
asset_valuation_engine: IAssetValuationEngine = FinvizAssetValuationEngine(html_data_access=html_data_access)
public_asset_data_access: IPublicAssetDataAccess = YahooFinanceDataAccess()
asset_calculations_engine: IAssetCalculationsEngine = SimpleAssetCalculationsEngine(public_asset_data_access=public_asset_data_access)
personal_asset_data_access: IPersonalAssetDataAccess = EasyEquitiesPersonalAssetDataAccess(resource_data_access=resource_data_access,
                                                                         username=None,
                                                                         password=None)
instance: IAssetProjectionEngine = SimpleAssetProjectionEngine(asset_valuation_engine=asset_valuation_engine,
                                                               public_asset_data_access=public_asset_data_access)

## Project for a Single Symbol

In [6]:
n_months: int = 12*30 # 39 year
usd_zar_exchange_rate: float = 16.17
symbol: str = 'AAPL'
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
initial_value: float = holdings_with_profits.loc[holdings_with_profits['symbol'] == 'AAPL']['total_current_usd'].sum()
monthly_deposit: float = 2000 / usd_zar_exchange_rate / len(holdings_with_profits)

growth: pd.DataFrame = instance.project_monthly_asset_growth(n_months=n_months,
                                                             symbol=symbol,
                                                             principal_value=initial_value,
                                                             monthly_deposit=monthly_deposit)

growth

,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
month,,,,,
0,0.000000,0.000000,0.000000,0.000000,74.697050
1,30.921459,0.611893,30.921459,0.611893,106.230403
2,30.921459,0.870204,61.842919,1.482097,138.022067
3,30.921459,1.150951,92.764378,2.633048,170.094477
4,30.921459,1.393357,123.685838,4.026405,202.409294
...,...,...,...,...,...
356,30.921459,546.736194,11008.039579,56237.894799,67320.631429
357,30.921459,561.379265,11038.961039,56799.274064,67912.932153
358,30.921459,556.320103,11069.882498,57355.594167,68500.173716


## Project for a Entire Holdings (Multiple Symbols)

In [7]:
# 30 years
n_months: int = 12*30
usd_zar_exchange_rate: float = 16.17
holdings: pd.DataFrame = personal_asset_data_access.get_personal_transactions()
holdings_with_profits: pd.DataFrame = asset_calculations_engine.calculate_holdings_profits(holdings=holdings)
# Simply R 1000 converted to USD and split evenly among how many assets are held.
monthly_deposits: list = [(2000 / usd_zar_exchange_rate / len(holdings_with_profits)) for h in holdings_with_profits]
holdings_growth = instance.project_monthly_holdings_growth(n_months=n_months,
                                                           holdings_with_profits=holdings_with_profits,
                                                           monthly_deposits=monthly_deposits)

holdings_growth

,month,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
0,1,123.685838,0.023089,123.685838,6.103659e+00,3.880418e+02
1,2,123.685838,0.022082,247.371676,1.486597e+01,5.204899e+02
2,3,123.685838,0.021668,371.057514,2.639366e+01,6.557035e+02
3,4,123.685838,0.021405,494.743352,4.073597e+01,7.937316e+02
4,5,123.685838,0.021295,618.429190,5.800612e+01,9.346876e+02
...,...,...,...,...,...,...
355,356,123.685838,0.041393,44032.158318,2.739205e+09,2.739249e+09
356,357,123.685838,0.041394,44155.844156,2.857490e+09,2.857534e+09
357,358,123.685838,0.041395,44279.529994,2.980886e+09,2.980930e+09
358,359,123.685838,0.041396,44403.215832,3.109613e+09,3.109658e+09


## Project Wealth Over Months

In [8]:
monthly_zar_deposit: float = 2000
usd_zar_exchange_rate: float = 16.17
monthly_deposits: list = [(monthly_zar_deposit / usd_zar_exchange_rate / len(holdings_with_profits)) for h in holdings_with_profits]
wealth_projection = instance.project_wealth_growth(holdings_with_profits=holdings_with_profits,
                                                   monthly_deposits=monthly_deposits)

wealth_projection

,Target Balance (USD),Target Balance Eng (USD),ETA (Months),ETA (Years)
0,1.078995e+03,one thousand and seventy-eight point nine nine...,5,0.416667
1,1.027494e+04,"ten thousand, two hundred and seventy-four poi...",43,3.583333
2,1.023456e+05,"one hundred and two thousand, three hundred an...",107,8.916667
3,1.014696e+06,"one million, fourteen thousand, six hundred an...",166,13.833333
4,1.015549e+07,"ten million, one hundred and fifty-five thousa...",222,18.500000
5,1.018406e+08,"one hundred and one million, eight hundred and...",277,23.083333
6,1.036325e+09,"one billion, thirty-six million, three hundred...",332,27.666667
7,1.016165e+10,"ten billion, one hundred and sixty-one million...",386,32.166667
